In [64]:
import pandas as pd


In [65]:
house_dataset = pd.read_csv('../data/ParisHousing.csv')
# house_dataset = pd.get_dummies(house_dataset, columns=['page_url', 'property_type', 'location', 'city', 'province_name', 'area', 'agency', 'Area Type', 'Area Category'])
house_dataset

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1726,89,0,1,5,73133,7,6,2009,0,1,9311,1698,218,0,4,176425.9
9996,44403,29,1,1,12,34606,9,4,1990,0,1,9061,1742,230,0,0,4448474.0
9997,83841,3,0,0,69,80933,10,10,2005,1,1,8304,7730,345,1,9,8390030.5
9998,59036,70,0,0,96,55856,1,3,2010,0,1,2590,6174,339,1,4,5905107.0


In [66]:
from sklearn.model_selection import train_test_split
X = house_dataset.copy()
X = X.drop(columns=['price'])
y = house_dataset['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [67]:
y_train_array = np.array(y_train.values)
y_train_array[0:3]

array([1682658.4, 6912274. , 6034513.5])

In [68]:
X_train_array = np.array(X_train.values)
X_train_array[0:3]

array([[16771,    48,     1,     0,    18, 41970,     6,     7,  2014,
            0,     1,   995,  5062,   884,     1,     8],
       [68999,    56,     1,     1,    38,  4761,     1,     8,  2020,
            1,     1,  7260,  7608,   437,     1,     3],
       [60244,    35,     1,     1,    47, 69892,     6,    10,  1995,
            1,     0,  1418,   923,   344,     0,     7]], dtype=int64)

In [69]:
X_test_array = np.array(X_test.values)
X_test_array[0:3]

array([[68270,    37,     1,     0,    65, 18486,     7,     9,  2018,
            0,     1,  3366,  2943,   928,     1,     0],
       [91543,     6,     1,     0,    77, 55027,     5,     9,  2013,
            1,     0,  1902,  9331,   211,     1,     0],
       [ 1066,     5,     1,     1,    46, 65388,     4,     9,  2017,
            0,     1,  2723,  9228,   524,     1,     5]], dtype=int64)

In [70]:
y_test_array = np.array(y_test.values)
y_test_array

array([6835596.8, 9166876.3,  114605.7, ..., 3620813.9, 5152982.7,
       6201486.8])

In [71]:
y_train

9501    1682658.4
7063    6912274.0
814     6034513.5
1566    1723749.4
8541    8609323.5
          ...    
6280    7159665.0
2368    8788809.4
9927     975297.5
1123    2892484.7
1916    2862521.9
Name: price, Length: 8000, dtype: float64

In [107]:
import numpy as np

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index 
        self.threshold = threshold
        self.left = left  
        self.right = right
        self.value = value  

class CustomDecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples=1):
        self.max_depth = max_depth
        self.min_samples = min_samples

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        variance = np.var(y)

        if depth == self.max_depth or n_samples <= self.min_samples or variance == 0:
            return Node(value=np.mean(y))

        best_variance_reduction = 0
        best_feature_index = None
        best_threshold = None
        for feature_index in range(n_features):
            thresholds, values = zip(*sorted(zip(X[:, feature_index], y)))
            for i in range(1, n_samples):
                if values[i-1] == values[i]:
                    continue  
                threshold = (thresholds[i-1] + thresholds[i]) / 2
                y_left = y[X[:, feature_index] <= threshold]
                y_right = y[X[:, feature_index] > threshold]
                variance_reduction = variance - (len(y_left) / n_samples * np.var(y_left) + len(y_right) / n_samples * np.var(y_right))
                if variance_reduction > best_variance_reduction:
                    best_variance_reduction = variance_reduction
                    best_feature_index = feature_index
                    best_threshold = threshold

        if best_variance_reduction == 0:
            return Node(value=np.mean(y))

        left_indices = X[:, best_feature_index] <= best_threshold
        right_indices = X[:, best_feature_index] > best_threshold
        left = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        return Node(feature_index=best_feature_index, threshold=best_threshold, left=left, right=right)

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree_) for x in X])

    def _predict_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._predict_tree(x, node.left)
        else:
            return self._predict_tree(x, node.right)


# X_train = np.array([[ 5340,    86,     0,     0,    44, 13348,     2,    10,  2011,
#             0,     0,  2155,  7363,   419,     1,     2],
#        [98689,    75,     1,     1,    55, 95076,     4,     4,  2009,
#             0,     0,  3462,  3915,   882,     0,     6],
#        [20666,    83,     1,     0,    57, 74967,     4,     1,  1994,
#             1,     1,  5379,  3285,   984,     0,     1]])
# y_train = np.array([ 538173.1, 9874062.9, 2072485.4])
# X_test = np.array([[  958,    48,     1,     0,    15, 41937,     7,     4,  2016,
#             1,     1,   575,  6655,   740,     1,     1],
#        [75749,    81,     1,     0,    30, 62511,     5,     3,  2020,
#             1,     0,  5132,  1182,   523,     1,     5],
#        [68528,    34,     1,     0,    78, 23274,     1,     1,  1997,
#             0,     1,  8731,  8777,   418,     1,     4]])


X_train = np.array([[1,11], [2,12], [3,13], [4,14], [5,15]])
y_train = np.array([2, 3, 4, 5, 5])
X_test = np.array([[2.5], [3.5]])


tree = DecisionTreeRegressor(max_depth=10, min_samples=20)
tree.fit(X_train_array, y_train_array)


predictions = tree.predict(X_test_array)
print("Predictions:", predictions)

results = pd.DataFrame({'Actual': y_test_array, 'Predicted': predictions})
print(results)

C:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\micha\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Predictions: [6834746.675      9154844.01428572  107583.02222222 ... 3616523.32307692
 5158391.49444444 6194304.04285714]
         Actual     Predicted
0     6835596.8  6.834747e+06
1     9166876.3  9.154844e+06
2      114605.7  1.075830e+05
3     8951286.3  8.955613e+06
4     4540276.8  4.528195e+06
...         ...           ...
1995  6428786.0  6.432384e+06
1996  8853221.2  8.859053e+06
1997  3620813.9  3.616523e+06
1998  5152982.7  5.158391e+06
1999  6201486.8  6.194304e+06

[2000 rows x 2 columns]


In [80]:
X_train = np.array([[1,2], [2,2], [3,2], [4,2], [5,2]])
X_train.shape

(5, 2)